In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Sabitler (Config gibi düşünülebilir)
DATA_PATH = r'../data/MiningProcess_Flotation_Plant_Database.csv'
MODEL_PATH = r'../models/final_xgboost_model.pkl'

def load_and_clean_data(path):
    print("1. Veri Yükleniyor...")
    df = pd.read_csv(path, decimal=',')
    # Sütun isim temizliği
    df.columns = [col.replace(' ', '_').replace('%_', '').replace('(', '').replace(')', '') for col in df.columns]
    # Tarih formatı
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.sort_values('date')
    return df

def feature_engineering(df):
    print("2. Özellik Mühendisliği (Lag & Rolling) Yapılıyor...")
    input_cols = [col for col in df.columns if col not in ['date', 'Silica_Concentrate']]

    for col in input_cols:
        # Son 5 periyodun ortalaması (Gürültü azaltma)
        df[f'{col}_Rolling_Mean'] = df[col].rolling(window=5).mean()
        # Bir önceki adımın değeri (Gecikme etkisi)
        df[f'{col}_Lag1'] = df[col].shift(1)

    df = df.dropna()
    return df

def train_model(df):
    print("3. Model Eğitiliyor (XGBoost)...")
    X = df.drop(['date', 'Silica_Concentrate'], axis=1)
    y = df['Silica_Concentrate']

    # Time Series Split (Shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False, random_state=42)

    # Optuna ile bulduğumuz en iyi parametreler (Önceki adımdan aldık)
    best_params = {
        'n_estimators': 814,
        'max_depth': 3,
        'learning_rate': 0.044,
        'subsample': 0.707,
        'colsample_bytree': 0.687,
        'reg_alpha': 2.66,
        'reg_lambda': 8.34
    }

    model = xgb.XGBRegressor(**best_params, random_state=42)
    model.fit(X_train, y_train)

    # Test Sonuçları
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"--- Model Başarısı ---\nRMSE: {rmse:.4f}\nR2 Score: {r2:.4f}")

    return model

def save_model(model, path):
    print(f"4. Model Kaydediliyor: {path}")
    # Klasör yoksa oluştur
    folder = os.path.dirname(path)
    if not os.path.exists(folder):
        os.makedirs(folder)

    joblib.dump(model, path)
    print("İşlem Tamamlandı! ✅")

if __name__ == "__main__":
    # Akışı Çalıştır
    df = load_and_clean_data(DATA_PATH)
    df = feature_engineering(df)
    final_model = train_model(df)
    save_model(final_model, MODEL_PATH)

1. Veri Yükleniyor...
2. Özellik Mühendisliği (Lag & Rolling) Yapılıyor...
3. Model Eğitiliyor (XGBoost)...
--- Model Başarısı ---
RMSE: 0.6457
R2 Score: 0.7028
4. Model Kaydediliyor: ../models/final_xgboost_model.pkl
İşlem Tamamlandı! ✅
